# Hilbert-Huang Transform (HHT) 解析チュートリアル

このチュートリアルでは、**gwexpy** を使用した **Hilbert-Huang Transform (HHT)** 解析の手順を解説します。
HHT は、非線形・非定常な信号の解析に適しており、従来のフーリエ変換やウェーブレット変換では捉えきれない局所的な周波数変動（瞬時周波数）を抽出するのに強力な手法です。

HHT は以下の2つの主要なステップから成ります：
1.  **Empirical Mode Decomposition (EMD)**: 信号を固有モード関数 (IMF: Intrinsic Mode Function) に分解します。
2.  **Hilbert Spectral Analysis**: 各 IMF に対してヒルベルト変換を行い、瞬時振幅と瞬時周波数を計算します。

**注意**: この機能を使用するには `PyEMD` (EMD-signal) パッケージがインストールされている必要があります。
```bash
pip install EMD-signal
```

In [1]:
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np

from gwexpy.timeseries import TimeSeries

# PyEMDの確認
try:
    import PyEMD

    print("PyEMD is installed and ready.")
except ImportError:
    raise ImportError(
        "このチュートリアルには 'PyEMD' (EMD-signal) が必要です。 pip install EMD-signal を実行してください。"
    )

PyEMD is installed and ready.


## 1. 理論的背景とHHTの利点

### HHT (Hilbert-Huang Transform) とは
HHTは、非線形・非定常な信号データを解析するための強力な手法です。
以下の2つの主要なプロセスから構成されます：

1.  **EMD (Empirical Mode Decomposition)**: 元の信号を、異なる周波数スケールを持つ固有モード関数 (IMF: Intrinsic Mode Function) の和に分解します。
2.  **HSA (Hilbert Spectral Analysis)**: 各IMFに対してヒルベルト変換を適用し、**瞬時周波数 (Instantaneous Frequency)** と瞬時振幅を計算します。

### 従来のSTFTとの違い
短時間フーリエ変換 (STFT) やウェーブレット変換では、時間分解能と周波数分解能の間にトレードオフ（不確定性原理による制限）が存在します。
一方、HHTは局所的な時間スケールに基づいて適応的に基底を生成するため、この制限を受けず、**高い時間・周波数分解能**を同時に達成可能です。

### EEMD (Ensemble EMD)
通常のEMDでは、断続的な信号やノイズによって、一つのIMFの中に異なる周波数成分が混じる「モード混合 (Mode Mixing)」が発生することがあります。
これを解決するために、**EEMD (Ensemble EMD)** が提案されています。
-   元の信号にホワイトノイズを加えたものを多数生成（アンサンブル）。
-   それぞれの信号に対してEMDを実行。
-   得られたIMFのアンサンブル平均をとることで、ノイズをキャンセルし、物理的に意味のあるモードを安定して抽出します。


## 2. シミュレーション：SASIモードの再現

超新星爆発（CCSN）の重力波解析において、**SASI (Standing Accretion Shock Instability)** と呼ばれる不安定性は重要な観測対象です。
ここでは、SASI由来の重力波を模擬した **Sine-Gaussian** 信号を作成し、解析してみましょう（参考: Takeda et al. 2021）。

Sine-Gaussian信号は以下の式で表されます：
$$ h(t) = A \exp\left( - \left( \frac{2\pi (t - t_0)}{t_{\text{width}}} \right)^2 \right) \sin(2\pi f_0 t) $$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from gwexpy.timeseries import TimeSeries

def sine_gaussian(t, t0, t_width, f0, amp=1.0):
    exponent = -((2 * np.pi * (t - t0)) / t_width)**2
    return amp * np.exp(exponent) * np.sin(2 * np.pi * f0 * t)

# パラメータ設定 (SASIモードを模倣)
duration = 0.5  # seconds (計算コスト削減のため短縮)
fs = 4096       # sampling rate
t = np.linspace(0, duration, int(duration * fs))

# 信号生成
t0 = 0.25       # peak time (center of 0.5s)
t_width = 0.4   # width
f0 = 130.0      # frequency (Hz) - SASI typical frequency
clean_signal = sine_gaussian(t, t0, t_width, f0, amp=1e-21)

# ノイズの追加
np.random.seed(42)
noise = np.random.normal(0, 0.5e-21, len(t))
data = clean_signal + noise

# TimeSeries オブジェクトの作成
ts_data = TimeSeries(data, t0=0, sample_rate=fs, name="Simulated SASI Signal", unit="strain")

# プロット
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(t, ts_data.value, label='Noisy Signal', color='lightgray')
ax.plot(t, clean_signal, label='Target Signal (SASI)', color='tab:blue')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Strain')
ax.set_xlim(0.5, 1.5)
ax.legend()
plt.title("Simulated SASI Gravitational Wave Signal")
plt.show()


## 3. 比較解析：STFT vs HHT

同じ信号に対して、STFT（スペクトログラム）と HHT（ヒルベルトスペクトル）を適用し、その分解能の違いを比較します。


In [ ]:
from scipy.signal import spectrogram, hilbert
import PyEMD

# --- 1. STFT (Spectrogram) ---
f_stft, t_stft, Sxx = spectrogram(ts_data.value, fs=fs, nperseg=256, noverlap=240)

# --- 2. HHT (EMD + Hilbert) ---
# EMDの実行
emd = PyEMD.EMD()
imfs = emd.emd(ts_data.value)
n_imfs = imfs.shape[0]

# ヒルベルト変換による瞬時周波数・振幅の計算
instant_freqs = []
instant_amps = []

for imf in imfs:
    analytic_signal = hilbert(imf)
    inst_amp = np.abs(analytic_signal)
    inst_phase = np.unwrap(np.angle(analytic_signal))
    inst_freq = (np.diff(inst_phase) / (2.0 * np.pi) * fs)
    
    # 配列サイズを合わせるために先頭要素を複製（diffで1つ減るため）
    inst_freq = np.insert(inst_freq, 0, inst_freq[0])
    
    instant_freqs.append(inst_freq)
    instant_amps.append(inst_amp)

# --- プロットによる比較 ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# STFT Plot
ax1.pcolormesh(t_stft, f_stft, Sxx, shading='gouraud', cmap='viridis')
ax1.set_ylabel('Frequency [Hz]')
ax1.set_xlabel('Time [sec]')
ax1.set_title('Short-Time Fourier Transform (STFT)')
ax1.set_ylim(0, 300)
ax1.set_xlim(0.5, 1.5)

# HHT Plot
# 散布図としてプロット (色を振幅に対応)
for i in range(n_imfs):
    # SASIモードに近い周波数帯(50-200Hz)のみ強調表示などの処理も可能
    # ここでは全てのIMFを表示
    sc = ax2.scatter(t, instant_freqs[i], c=instant_amps[i], cmap='viridis', 
                     s=2, alpha=0.5, vmin=0, vmax=np.max(instant_amps)/2)

ax2.set_ylabel('Instantaneous Frequency [Hz]')
ax2.set_xlabel('Time [sec]')
ax2.set_title('Hilbert-Huang Transform (HHT)')
ax2.set_ylim(0, 300)
ax2.set_xlim(0.5, 1.5)

# カラーバーの追加
cbar = plt.colorbar(sc, ax=ax2)
cbar.set_label('Instantaneous Amplitude')

plt.tight_layout()
plt.show()


## 4. 応用：EEMDによるモード抽出とパラメータ推定

### EEMDの適用
上記の単純なEMDの結果では、ノイズの影響で周波数が振動したり、本来の130Hzの信号が複数のIMFに分割されてしまう（モード混合）ことがあります。
**EEMD (Ensemble EMD)** を使用することで、これを改善できます。

> **Sasaoka et al. (2024)** では、CCSNからの重力波解析において、EEMD（またはCEEMD）を用いることで、原始中性子星（PNS）の振動モードを正確に抽出し、その周波数進化から PNS の物理量（質量や半径の比 $M/R^2$）を推定できることが示されています。

以下のコードは、PyEMDを使用してEEMDを実行する例です（計算時間がかかるため、試行回数 `trials` を少なめに設定しています）。


In [ ]:
# EEMDの実行 (計算負荷軽減のため、trials=10程度、データ長も短めに設定することを推奨)
from PyEMD import EEMD

# データの正規化（数値安定性のため推奨）
data_std = np.std(ts_data.value)
data_norm = ts_data.value / data_std

# EEMDのインスタンス化
# noise_width: 追加するノイズの標準偏差（0.2 = 信号標準偏差の20%）
eemd = EEMD(trials=10, noise_width=0.2)

# 実行
print("Running EEMD... please wait.")
eimfs_norm = eemd.eemd(data_norm)
eimfs = eimfs_norm * data_std # スケールを戻す
n_eimfs = eimfs.shape[0]
print(f"EEMD completed. Extracted {n_eimfs} IMFs.")

# 特定のIMFの可視化
fig, axes = plt.subplots(n_eimfs, 1, figsize=(10, 2 * n_eimfs), sharex=True)
if n_eimfs == 1:
    axes = [axes] # 配列化してインデックスアクセス可能にする

for i in range(n_eimfs):
    axes[i].plot(t, eimfs[i], 'k')
    axes[i].set_ylabel(f'IMF {i+1}')
    if i == n_eimfs - 1:
        axes[i].set_xlabel('Time [s]')

plt.suptitle("EEMD Decomposed Modes")
plt.tight_layout()
plt.show()

# 解説：SASIモードの特定
# 通常、信号の主要なエネルギーを持つモードは低次（IMF 2-4あたり）に現れます。
# ヒルベルトスペクトルを再度描画して、130Hzの線がより鮮明になっているか確認してみてください。
